In [1]:
import os
import time

import torch
import shutil
import numpy as np
import torch.nn as nn

from pathlib import Path
import pytorch_lightning as pl
from dataclasses import dataclass
from dataclasses import dataclass,asdict, field
from typing import Any, Dict, Optional, Union, List,Tuple

from pytorch_lightning import Trainer
from torch.utils.data import Dataset, DataLoader
from lightning.pytorch.loggers import MLFlowLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from fimodemix.configs.config_classes.fim_sde_config import FIMSDEModelParams
from fimodemix.data.dataloaders import (
    FIMSDEpDataLoader
)
from fimodemix.utils.experiment_files import ExperimentsFiles
from fimodemix.models.fim_sde import FIMSDEp

In [2]:
# Experiment
experiment_files = ExperimentsFiles(experiment_indentifier=None,
                                    delete=True)
# Set up TensorBoard logger
logger = MLFlowLogger(experiment_name="time_series_transformer",
                      tracking_uri="http://localhost:5000",
                      save_dir="mlruns")

# Set up Model Checkpointing
checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath=experiment_files.checkpoints_dir,
    filename='best-checkpoint-hey',
    save_top_k=1,
    mode='min',
    save_weights_only=True,
    every_n_train_steps=100  # Save checkpoint every 100 training steps
)


In [3]:
params = FIMSDEModelParams()

#Set up Dataloaders
dataloaders = FIMSDEpDataLoader(params)

# Set up Model
model = FIMSDEp(params)

#Set up trainers
trainer = Trainer(
    max_epochs=params.num_epochs,
    logger=logger,
    callbacks=[checkpoint_callback]
)
trainer.fit(model, dataloaders.train_it)

c:\users\cesar\desktop\projects\foundationmodels\fimodemix\src\fimodemix\data\datasets.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data: FIMSDEpDatabatch = torch.lo

Max Hypercube Size: 1024
Max Dimension: 3
Max Num Steps: 129
Max Hypercube Size: 1024
Max Dimension: 3
Max Num Steps: 129


c:\Users\cesar\anaconda3\envs\fimode\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Max Hypercube Size: 1024
Max Dimension: 3
Max Num Steps: 129


Experiment with name time_series_transformer not found. Creating it.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | phi_t0    | TimeEncoding       | 38     | train
1 | phi_x0    | Mlp                | 3.8 K  | train
2 | phi_1     | Mlp                | 2.9 K  | train
3 | phi_2     | Mlp                | 4.8 K  | train
4 | psi1      | TransformerModel   | 93.2 K | train
5 | query_1x  | QueryGenerator     | 160    | train
6 | query_1   | StaticQuery        | 5.2 K  | train
7 | omega_1   | MultiheadAttention | 6.6 K  | train
8 | criterion | MSELoss            | 0      | train
---------------------------------------------------------
116 K     Trainable params
0         Non-trainable params
116 K     Total params
0.466     Total estimated model params size (MB)
61        Modules in train mode
0         Modules in eval mode
c:\Users\cesar\anaconda3\envs\fimode\Lib\site-packages\pyto

...

c:\Users\cesar\anaconda3\envs\fimode\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch 9: 100%|██████████| 313/313 [00:04<00:00, 74.97it/s, v_num=def6]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 313/313 [00:04<00:00, 74.94it/s, v_num=def6]


2024/10/15 14:52:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run youthful-fox-866 at: http://localhost:5000/#/experiments/791522484725792022/runs/76769b9823364aa7bab16466881edef6.
2024/10/15 14:52:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/791522484725792022.


In [4]:
# mlflow.log_params(params)